In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("airfoil_noise_data.csv")
df.head()

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [3]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [4]:
X_train.shape,Y_train.shape

((1202, 5), (1202, 1))

In [5]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [6]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None):
        self.root = None
        # stopping conditions for regularization
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats=n_feats
    
    def fit(self,X,y):
        dataset = np.concatenate((X,y),axis=1)
        self.root = self.build_tree(dataset)
    
    def variance_reduction(self,parent,left,right):
        weight_l = len(left)/len(parent)
        weight_r = len(right)/len(parent)
        var_red = np.var(parent)-(weight_l*np.var(left)+weight_r*np.var(right))
        return var_red
    
    def calculate_leaf_value(self, y):
        mean_y = np.mean(y)
        return mean_y
        
    def build_tree(self,dataset,curr_depth=0):
        x = dataset[:,:-1]
        y = dataset[:,-1]
        n_labels = len(np.unique(y))
        num_examples,num_features=x.shape
        if curr_depth<=self.max_depth and num_features>=self.min_samples_split:
            if self.n_feats is None:
                feat_idxs = np.arange(num_features)
            else:
                feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)
            best_split=self.get_best_split(dataset,num_examples,feat_idxs)
            if best_split!={} and best_split["var_red"]>0:
                feature_index=best_split["feature_index"]
                threshold=best_split["threshold"]
                left=best_split["left"] 
                right=best_split["right"]
                var_red=best_split["var_red"]
                left_subtree=self.build_tree(left,curr_depth+1)
                right_subtree=self.build_tree(right,curr_depth+1)
                return Node(feature_index,threshold,left_subtree,right_subtree,var_red)
        leaf_val=self.calculate_leaf_value(y)
        return Node(value=leaf_val)

    def get_best_split(self,dataset,num_examples,feat_idxs):
        best_split = {}
        y = dataset[:,-1]
        max_var_red= -float("inf")
        for feature_index in feat_idxs:
            feature_vector = dataset[:,feature_index]
            thresholds = np.unique(feature_vector)
            for threshold in thresholds:
                dataset_left,dataset_right = self.split(dataset,feature_index,threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y_left, y_right = dataset_left[:,-1],dataset_right[:,-1]
                    curr_var_red = self.variance_reduction(y,y_left,y_right)
                    if max_var_red<curr_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left"] =dataset_left
                        best_split["right"]=dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red =curr_var_red
        return best_split
    
    def split(self,dataset,feature_index,threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def print_tree(self,tree=None, indent=" "):
        
        if tree==None:
            tree=self.root
        
        if tree.value is None:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "? info_gain: ", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        else:
            print(tree.value)
    
    def predict(self,x):
        
        preds = np.array([self.make_prediction(xi,self.root) for xi in x])
        return preds
    
    def make_prediction(self,x,tree):
        if tree.value is not None:
            return tree.value
        threshold = tree.threshold
        if x[tree.feature_index] <= threshold:
            return self.make_prediction(x,tree.left)
        else:
            return self.make_prediction(x,tree.right)


In [7]:
my_regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
my_regressor.fit(X_train,Y_train)
my_regressor.print_tree()

X_0 <= 3150.0 ? info_gain:  7.132048702017748
 left:X_4 <= 0.0337792 ? info_gain:  3.590330569067664
  left:X_3 <= 55.5 ? info_gain:  1.17898999813184
    left:X_4 <= 0.00251435 ? info_gain:  1.614396721819876
        left:128.9919833333333
        right:125.90953579676673
    right:X_1 <= 15.4 ? info_gain:  2.2342245360792994
        left:129.39160280373832
        right:123.80422222222222
  right:X_0 <= 1250.0 ? info_gain:  9.970884020498868
    left:X_4 <= 0.0483159 ? info_gain:  6.35527515982486
        left:124.38024528301887
        right:118.30039999999998
    right:X_3 <= 39.6 ? info_gain:  5.036286657241031
        left:113.58091666666667
        right:118.07284615384616
 right:X_4 <= 0.00146332 ? info_gain:  29.08299210506528
  left:X_0 <= 8000.0 ? info_gain:  11.886497073996964
    left:X_2 <= 0.0508 ? info_gain:  7.608945827689519
        left:134.04247500000002
        right:127.33581818181818
    right:X_4 <= 0.00076193 ? info_gain:  10.6229193224008
        left:128.9407

In [8]:
Y_pred = my_regressor.predict(X_test) 
np.sqrt(mean_squared_error(Y_test, Y_pred))

4.851358097184457

## Sklearn 

In [20]:
sklearn_clf = DecisionTreeRegressor(random_state=0,min_samples_split=3, max_depth=3)
sklearn_clf.fit(X_train,Y_train)
sklearn_preds=sklearn_clf.predict(X_test)
np.sqrt(mean_squared_error(Y_test, sklearn_preds))

5.376979589924685